In [ ]:
import requests
APIKEY = "b0cde630-ce66-11e8-951c-b3d75228cc98"
USERNAME = "cdc43339"

In [ ]:
# Changing request
parameters = {
    "size":10,
    "apikey":APIKEY,
    "sort": "totalpageviews",
    "sortorder": "desc"
}
R = requests.get(url,params=parameters)
R.json()

In [ ]:
# Van Gogh Exercise
personurl = "https://api.harvardartmuseums.org/person"
van_gogh_parameters = {
    "apikey":APIKEY,
    "q":"displayname:Van Gogh",
    "size":10

}
R = requests.get(personurl,params=van_gogh_parameters)
for record in R.json()['records']:
    print("{}: {}".format(record['alphasort'], record['id']))
    
van_gogh_works_params = {
    "apikey": APIKEY,
    "person": 22730,
    "size": 100,
    "hasimage": 1
}
R = requests.get(url, params=van_gogh_works_params)
for record in R.json()['records']:
    print("{} - {}".format(record['title'],record['url']))

In [ ]:
def ham_query(apikey, endpoint, **kwargs):
    """Sends kwargs to the specified endpoint, using apikey for authentication"""
    params = kwargs
    params['apikey'] = apikey
    url = "https://api.harvardartmuseums.org/{}".format(endpoint)
    R = requests.get(url,params=params)
    return R

In [ ]:
def ham_query_with_id(apikey, endpoint, ID, **kwargs):
    """Sends kwargs to the specified endpoint, using apikey for authentication. ID is a required arg (appears in the route)"""
    params = kwargs
    params['apikey'] = apikey
    url = "https://api.harvardartmuseums.org/{}/{}".format(endpoint,ID)
    R = requests.get(url,params=params)
    return R

In [ ]:
response = ham_query(APIKEY, "exhibition", status="current", size=100)
current_exhibits = response.json()['records']

In [ ]:
for ex in current_exhibits:
    exhibit = ham_query(APIKEY, "object", exhibition=ex['id'], size=100).json()
    total_peoplecount = 0
    for record in exhibit['records']:
        total_peoplecount += record['peoplecount']
    print("{} - id: {}, total peoplecount: {}".format(ex['title'], ex['id'], total_peoplecount))

In [ ]:
peoplecounts = []
for ex in current_exhibits:
    exhibit = ham_query(APIKEY, "object", exhibition=ex['id'], size=100).json()
    total_peoplecount = 0
    for record in exhibit['records']:
        total_peoplecount += record['peoplecount']
    peoplecounts.append({
        "id": ex['id'],
        "title": ex['title'],
        "peoplecount": total_peoplecount
    })
peoplecounts.sort(key = lambda x: x['peoplecount'], reverse=True)

for ex in peoplecounts:
    print("{} - id: {}, total peoplecount: {}".format(ex['title'], ex['id'], ex['peoplecount']))

It looks like "Winslow Homer: Eyewitness" and "Crossing Lines, Constructing Home: Displacement and Belonging in Contemporary Art" are our best candidates.

In [ ]:
homer = ham_query(APIKEY, "object", exhibition=5660, size=200).json()
homer['exhibition_info'] = ham_query_with_id(APIKEY, endpoint="exhibition", ID=5660).json()

crossing_lines = ham_query(APIKEY, "object", exhibition=5834, size=200).json()
crossing_lines['exhibition_info'] = ham_query_with_id(APIKEY, endpoint="exhibition", ID=5834).json()

In [ ]:
def search_place(placename):
    url = "http://api.geonames.org/searchJSON"
    Q = {
        'q':placename,
        'username':USERNAME,
    }
    results = requests.get(url,params=Q).json()
    top_result = results['geonames'][0]
    return top_result['lat'], top_result['lng']

In [ ]:
def get_people_from_exhibit(exhibit):
    exhibit_people = {}
    for record in exhibit['records']:
        if record['peoplecount'] > 0:
            obj = record['people']
        for person in obj:
            if person['birthplace'] is not None:
                person['birthplace_coordinates'] = search_place(person['birthplace'])
            if person['deathplace'] is not None:
                person['deathplace_coordinates'] = search_place(person['deathplace'])
        exhibit_people[record['id']] = obj
    return exhibit_people

crossing_lines['people'] = get_people_from_exhibit(crossing_lines)
homer['people'] = get_people_from_exhibit(homer)

print(crossing_lines['people'])

# crossing_lines_people = get_people_from_exhibit(crossing_lines)
# homer_people = get_people_from_exhibit(homer)
             
# for record in crossing_lines['records']:
# #     if 'peoplecount' in record:
# #         print(record['peoplecouht'])
#     if record['peoplecount'] > 0:
#         obj = record['people']
#         for person in obj:
#             if person['birthplace'] is not None:
#                 person['birthplace_coordinates'] = search_place(person['birthplace'])
#             if person['deathplace'] is not None:
#                 person['deathplace_coordinates'] = search_place(person['deathplace'])
#         people[record['id']] = obj

In [ ]:
for painting in crossing_lines['people']:
    people = crossing_lines['people'][painting]
#     print(people)
    for person in people:
        print(person['alphasort'])

In [ ]:
# from ipyleaflet import *
# m = Map(zoom=8, basemap=basemaps.Esri.DeLorme)

# # print(people)

# birthicon = Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-green.png', icon_size=[38, 95], icon_anchor=[22,94])

# for painting_id in people:
# #     print(people[person]['birthplace_coordinates'])
#     p = people[painting_id]
#     for person in p:
#         if 'birthplace_coordinates' in person:
#             center = person['birthplace_coordinates']
#             marker = Marker(location=center, icon=birthicon)
#             m.add_layer(marker)
# # print(person['birthplace_coordinates'])
# #     print(type(p['birthpace_coordinates']))
# #     center = p['birthplace_coordinates']
# #    print(center)
# #     marker = Marker(location=center)
# #     m.add(marker)

# # center = (52.204793, 360.121558)
# # m = Map(center=center, zoom=15)
# # marker = Marker(location=center, draggable=False)
# # m.add_layer(marker);

# m

In [ ]:
def add_people_to_map(people, leaflet_map, birth_color="green", death_color="red", exhibit_name=None):
    for painting_id in people:
        p = people[painting_id]
        for person in p:
            if 'birthplace_coordinates' in person:
                if exhibit_name is not None:
                    label = '<h4>{}</h4><div style="font-weight:bold">{}</div>{}, {}'.format(exhibit_name, person['displayname'], person['displaydate'], person['birthplace'])
                else:
                    label = '<div style="font-weight:bold">{}</div>{}, {}'.format(person['displayname'], person['displaydate'], person['birthplace'])
                folium.Marker(
                    location = person['birthplace_coordinates'],
                    popup = label,
                    icon=folium.Icon(color=birth_color)
                ).add_to(leaflet_map)
            if 'deathplace_coordinates' in person:
                if exhibit_name is not None:
                    label = '<h4>{}</h4><div style="font-weight:bold">{}</div>died {}'.format(exhibit_name, person['displayname'], person['deathplace'])
                else:
                    label = '<div style="font-weight:bold">{}</div>died {}'.format(person['displayname'], person['deathplace'])
                folium.Marker(
                    location = person['deathplace_coordinates'],
                    popup = label,
                    icon=folium.Icon(color=death_color)
                ).add_to(leaflet_map)

add_people_to_map(crossing_lines['people'], people_map, birth_color="blue", death_color="orange", exhibit_name=crossing_lines['exhibition_info']['title'])
add_people_to_map(homer['people'], people_map, exhibit_name=homer['exhibition_info']['title'])
people_map
# for painting_id in people:
#     p = people[painting_id]
#     for person in p:
#         if 'birthplace_coordinates' in person:
#             label = '<div style="font-weight:bold">{}</div>{}, {}'.format(person['displayname'], person['displaydate'], person['birthplace'])
#             folium.Marker(
#                 location = person['birthplace_coordinates'],
#                 popup = label,
#                 icon=folium.Icon(color='green')
#             ).add_to(map)
#         if 'deathplace_coordinates' in person:
#             label = '<div style="font-weight:bold">{}</div> {}'.format(person['displayname'], person['deathplace'])
#             folium.Marker(
#                 location = person['deathplace_coordinates'],
#                 popup = label,
#                 icon=folium.Icon(color='red')
#             ).add_to(map)